In [14]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [15]:
class ParkinsonLoader():
  def __init__(self, path):
    parkinson_dataset = pd.read_csv('parkinsons_updrs.csv')
    # drop rows that have NAN value.
    parkinson_dataset = parkinson_dataset.dropna()
    # split x and y sub-dataset from the total dataset.
    columns = ['motor_UPDRS', 'total_UPDRS']
    parkinson_x = np.array(parkinson_dataset.drop(columns, axis=1))
    parkinson_y = np.array(pd.DataFrame(parkinson_dataset, columns=columns))
    # split the train and test dataset.
    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(parkinson_x, parkinson_y, test_size=0.25, random_state=5)
    # feature scaling uses the StandardScaler.
    self.scaler = preprocessing.StandardScaler().fit(self.X_train)
    # get the number of trainset and testset.
    self.num_train_data = np.shape(self.X_train)[0]
    self.num_test_data = np.shape(self.X_test)[0]

  def get_train_batch(self, batch_size):
    # randormly pick train batch (batch_size).
    index = np.random.randint(0, self.num_train_data, batch_size)
    return self.scaler.transform(self.X_train[index, :]), self.y_train[index, :]
  
  def get_testset(self):
    # get the test dataset.
    return self.scaler.transform(self.X_test), self.y_test

In [16]:
import tensorflow as tf
tf.keras.backend.set_floatx('float64')

class MLP(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(100, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(100, activation=tf.nn.relu)
    self.dense3 = tf.keras.layers.Dense(2)

  def call(self, inputs):
    x = np.array(inputs).reshape(1, len(inputs))
    x = self.dense1(x)
    x = self.dense2(x)
    output = self.dense3(x)
    return output

  def predict(self, testdata):
    return self.call(testdata)

In [19]:
import time
import progressbar

widgets=[
    ' [', progressbar.Timer(), '] ',
    progressbar.Bar(),
    ' (', progressbar.SimpleProgress(),') ',
    progressbar.Percentage(),
    ' (', progressbar.ETA(), ') ',
]

num_epochs = 400
batch_size = 100
learning_rate = 0.001
parkinson_path = "C:/Users/Jincy Joy/Desktop/ParkinsonDetection/parkinsons_updrs.csv"

model = MLP()
data_loader = ParkinsonLoader(parkinson_path)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
progress_bar = progressbar.ProgressBar(maxval=num_batches, widgets=widgets)

for batch_index in range(num_batches):
  loss = 0
  X, y = data_loader.get_train_batch(batch_size)
  with tf.GradientTape() as tape:
      for index in range(batch_size):
        y_pred = model(X[index, :])
        loss_temp = tf.losses.mean_squared_error(y_pred, y[index, :])
        loss_temp = tf.reduce_mean(loss_temp)
        loss = loss + loss_temp
  time.sleep(0.1)
  progress_bar.start()
  progress_bar.update(batch_index)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables))
progress_bar.finish()

 [Elapsed Time: 0:00:00] |#############| (17600 of 17600) 100% (Time: 0:00:00) 


In [21]:
accuracy = np.array([0, 0])
test_X, test_y = data_loader.get_testset()
for index in range(data_loader.num_test_data):
  y_pred = model.predict(test_X[index, :])
  accuracy = accuracy + abs(y_pred-test_y[index, :])
accuracy = accuracy/data_loader.num_test_data
print("MAE of motor_UPDRS: %f \nMAE of total_UPDRS: %f" % (accuracy[0][0], accuracy[0][1]))

MAE of motor_UPDRS: 0.662164 
MAE of total_UPDRS: 0.789696
